In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum as _sum
from pyspark.sql.window import Window
spark = SparkSession.builder.appName("Cumulative Score").getOrCreate()
deliveries_df = spark.read.option("header", True).csv("/FileStore/tables/deliveries.csv")

deliveries_df.show(5)


+--------+------+--------------------+--------------------+----+----+-----------+-------+-----------+------------+----------+----------+-----------+---------+----------------+--------------+-------+
|match_id|inning|        batting_team|        bowling_team|over|ball|     batter| bowler|non_striker|batsman_runs|extra_runs|total_runs|extras_type|is_wicket|player_dismissed|dismissal_kind|fielder|
+--------+------+--------------------+--------------------+----+----+-----------+-------+-----------+------------+----------+----------+-----------+---------+----------------+--------------+-------+
|  335982|     1|Kolkata Knight Ri...|Royal Challengers...|   0|   1| SC Ganguly|P Kumar|BB McCullum|           0|         1|         1|    legbyes|        0|              NA|            NA|     NA|
|  335982|     1|Kolkata Knight Ri...|Royal Challengers...|   0|   2|BB McCullum|P Kumar| SC Ganguly|           0|         0|         0|       null|        0|              NA|            NA|     NA|
|  33

In [0]:
from pyspark.sql.functions import col
window_spec = Window.partitionBy("match_id", "inning").orderBy(col("over").cast("int"), col("ball").cast("int")).rowsBetween(Window.unboundedPreceding, Window.currentRow)
deliveries_with_cumulative = deliveries_df.withColumn(
    "cumulative_score",
    _sum(col("total_runs").cast("int")).over(window_spec)
)

deliveries_with_cumulative.select("match_id", "inning", "over", "ball", "total_runs", "cumulative_score").show(20, truncate=False)


+--------+------+----+----+----------+----------------+
|match_id|inning|over|ball|total_runs|cumulative_score|
+--------+------+----+----+----------+----------------+
|1082593 |2     |0   |1   |1         |1               |
|1082593 |2     |0   |2   |0         |1               |
|1082593 |2     |0   |3   |1         |2               |
|1082593 |2     |0   |4   |0         |2               |
|1082593 |2     |0   |5   |1         |3               |
|1082593 |2     |0   |6   |4         |7               |
|1082593 |2     |1   |1   |1         |8               |
|1082593 |2     |1   |2   |6         |14              |
|1082593 |2     |1   |3   |1         |15              |
|1082593 |2     |1   |4   |4         |19              |
|1082593 |2     |1   |5   |0         |19              |
|1082593 |2     |1   |6   |1         |20              |
|1082593 |2     |2   |1   |4         |24              |
|1082593 |2     |2   |2   |1         |25              |
|1082593 |2     |2   |3   |1         |26        